Welcome, to Punisher. After running your strategy, view the live results with Dash

```
$ python -m punisher.charts.dash_charts.dash_record --name default_backtest
```

## TODO

* Wrap all Exchange Order responses in an Order object
* Market orders do not return price. So fetch trades. https://github.com/ccxt/ccxt/issues/798
* Verify starting_cash <= actual balance on Live Exchange
* Handle/prevent 'insufficient funds' from Binance/Poloniex (Above will help, but still need to catch if the exchange balance has changed for some reason)
* Handle/prevent 'minimum order not met' from Binanace/Poloniex (method to refresh and lookup the minimum amount)
* Method to audit/confirm our local exchange balance reflects actual changes to CCXT exchange balance
* Limit Sell Orders not working I think..


## Define Strategy

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from punisher.common import *

from punisher.strategies.strategy import Strategy

class SimpleStrategy(Strategy):
    def __init__(self):
        super().__init__()

    def log_all(self, orders, data, ctx, time_utc):
        self.logger = ctx.logger
        if self.logger is not None:
            self.log_epoch_time(time_utc)
            self.log_ohlcv(data)
            self.log_orders(orders)
            self.log_performance(ctx)
            self.log_balance(ctx)
            self.log_positions(ctx)
            self.log_metrics(ctx)

    def handle_data(self, data, ctx):
        orders = []
        asset = Asset(c.ETH, c.BTC)
        price = data['close']
        quantity = .01

        if random.random() > 0.5:
            order = order_manager.build_limit_buy_order(
                ctx.exchange, asset, quantity, price)
            orders.append(order)
        elif (len(ctx.record.portfolio.positions) > 0 
              and ctx.record.balance.get(c.ETH)[BalanceType.FREE] > 0.0):
            order = order_manager.build_market_sell_order(
                ctx.exchange, asset, quantity)
            orders.append(order)

        # Optionally cancel pending orders (LIVE trading)
        #pending_orders = ctx.exchange.fetch_open_orders(asset)
        cancel_ids = []
        
        # Add Metrics and OHLCV to Record
        self.update_metric('SMA', 5.0, ctx)
        self.update_metric('RSI', 10.0, ctx)
        self.update_ohlcv(data, ctx)

        self.log_all(orders, data, ctx, data['time_utc'])
        return {
            'orders': orders,
            'cancel_ids': cancel_ids
        }

## Backtest

In [ ]:
cash = 1.0
ohlcv_fpath = os.path.join(cfg.DATA_DIR, c.DEFAULT_30M_FEED_CSV_FILENAME)
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.THIRTY_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
feed = CSVDataFeed(
    fpath=ohlcv_fpath,
    start=None, # Usually None for backtest, but its possible to filter the csv
    end=None
)
exchange = load_feed_based_paper_exchange(balance, feed)
strategy = SimpleStrategy()
experiment_name = 'default_backtest'

In [ ]:
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

## Simulate Orders

In [4]:
cash = 1.0
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, c.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
assets = [Asset(c.ETH, c.BTC)]
feed = ExchangeDataFeed(
    exchange=exchange,
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy()
experiment_name = 'default_simulate'

In [ ]:
runner.simulate(experiment_name, exchange, balance, portfolio, feed, strategy)

Downloading: ETH/BTC
Downloaded rows: 1
Downloading: ETH/BTC
Downloaded rows: 1
No data after prior poll: 2018-01-18 07:21:22.737767
Downloading: ETH/BTC
Downloaded rows: 1
No data after prior poll: 2018-01-18 07:21:22.737767
Downloading: ETH/BTC
Downloaded rows: 2


---------------------------------------
Epoch 1 - Timestep: 2018-01-18T07:23:00
---------------------------------------
OHLCV
    O: 0.0900 | C: 0.0900 | V: 103.9 | T: 2018-01-18T07:23:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0900 | Quantity: 0.0100
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 0.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00


Placing Order {
    "id": "3fc542adae5f4ea884e342f1ad2a90b0",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.090004,
    "quantity": 0.01,
    "filled_quantity": 0.0,
    "order_type": "LIMIT_BUY",
    "status": "CREATED",
    "created_time": "2018-01-18T07:23:26.158743",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Placed order {
    "id": "3fc542adae5f4ea884e342f1ad2a90b0",
    "exchange_id": "paper",
    "exchange_order_id": "bc10318916a142f9a76f06d15ca5f54f",
    "price": 0.090004,
    "quantity": 0.01,
    "filled_quantity": 0.01,
    "order_type": "LIMIT_BUY",
    "status": "FILLED",
    "created_time": "2018-01-18T07:23:26.158743",
    "opened_time": null,
    "filled_time": "2018-01-18T07:23:26.298961",
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Downloading: ETH/BTC
Downloaded rows: 1

---------------------------------------
Epoch 2 - Timestep: 2018-01-18T07:24:00
---------------------------------------
OHLCV
    O: 0.0900 | C: 0.0898 | V: 117.0 | T: 2018-01-18T07:24:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0898 | Quantity: 0.0100
PERFORMANCE
    Cash: 0.9991 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.99819992, 'used': 0.0, 'total': 0.99819992}
    ETH - {'free': 0.02, 'used': 0.0, 'total': 0.02}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.01, 'cost_price': 0.090004, 'latest_price': 0.090004}
METRICS
    SMA: 5.00
    RSI: 10.00


Placing Order {
    "id": "620057fae1dc482eb8411867f4f630f9",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.0898,
    "quantity": 0.01,
    "filled_quantity": 0.0,
    "order_type": "LIMIT_BUY",
    "status": "CREATED",
    "created_time": "2018-01-18T07:24:26.733324",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Placed order {
    "id": "620057fae1dc482eb8411867f4f630f9",
    "exchange_id": "paper",
    "exchange_order_id": "864d3041758e4852a2acde401cf1c86a",
    "price": 0.0898,
    "quantity": 0.01,
    "filled_quantity": 0.01,
    "order_type": "LIMIT_BUY",
    "status": "FILLED",
    "created_time": "2018-01-18T07:24:26.733324",
    "opened_time": null,
    "filled_time": "2018-01-18T07:24:26.788300",
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Downloading: ETH/BTC
Downloaded rows: 1
No 

...

Placing Order {
    "id": "8e91f63d942f4f119321c43db5cd87f1",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.08951100000000001,
    "quantity": 0.01,
    "filled_quantity": 0.0,
    "order_type": "LIMIT_BUY",
    "status": "CREATED",
    "created_time": "2018-01-18T07:34:01.687276",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Placed order {
    "id": "8e91f63d942f4f119321c43db5cd87f1",
    "exchange_id": "paper",
    "exchange_order_id": "adfb30e0a27f4e98a7a7d54a793188dc",
    "price": 0.08951100000000001,
    "quantity": 0.01,
    "filled_quantity": 0.01,
    "order_type": "LIMIT_BUY",
    "status": "FILLED",
    "created_time": "2018-01-18T07:34:01.687276",
    "opened_time": null,
    "filled_time": "2018-01-18T07:34:01.740900",
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Downloading: ETH/

---------------------------------------
Epoch 13 - Timestep: 2018-01-18T07:35:00
---------------------------------------
OHLCV
    O: 0.0895 | C: 0.0896 | V: 129.7 | T: 2018-01-18T07:35:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0100
PERFORMANCE
    Cash: 0.9946 Total Val: 1.0000 PnL: -0.0000 Returns: -0.0000
BALANCE
    BTC - {'free': 0.9892268200000001, 'used': 0.0, 'total': 0.9892268200000001}
    ETH - {'free': 0.11999999999999998, 'used': 0.0, 'total': 0.11999999999999998}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.060000000000000005, 'cost_price': 0.08968683333333334, 'latest_price': 0.08951100000000001}
METRICS
    SMA: 5.00
    RSI: 10.00


Placing Order {
    "id": "874764f2c5084b85a36e6da46b99ea34",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.01,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "CREATED",
    "created_time": "2018-01-18T07:35:32.322993",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Placed order {
    "id": "874764f2c5084b85a36e6da46b99ea34",
    "exchange_id": "paper",
    "exchange_order_id": "c3fcd4e7c4084d8fa643292cf20bf389",
    "price": 0.089503,
    "quantity": 0.01,
    "filled_quantity": 0.01,
    "order_type": "MARKET_SELL",
    "status": "FILLED",
    "created_time": "2018-01-18T07:35:32.322993",
    "opened_time": null,
    "filled_time": "2018-01-18T07:35:32.544028",
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Downloading: ETH/BTC
Downloaded rows: 2


---------------------------------------
Epoch 14 - Timestep: 2018-01-18T07:36:00
---------------------------------------
OHLCV
    O: 0.0895 | C: 0.0895 | V: 3.4 | T: 2018-01-18T07:36:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0100
PERFORMANCE
    Cash: 0.9955 Total Val: 1.0000 PnL: -0.0000 Returns: -0.0000
BALANCE
    BTC - {'free': 0.9910168800000001, 'used': 0.0, 'total': 0.9910168800000001}
    ETH - {'free': 0.09999999999999999, 'used': 0.0, 'total': 0.09999999999999999}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.05, 'cost_price': 0.08968683333333334, 'latest_price': 0.08962}
METRICS
    SMA: 5.00
    RSI: 10.00


Placing Order {
    "id": "11239b80b8794301bb9f8cd31fdbb879",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.01,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "CREATED",
    "created_time": "2018-01-18T07:36:02.736890",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Placed order {
    "id": "11239b80b8794301bb9f8cd31fdbb879",
    "exchange_id": "paper",
    "exchange_order_id": "69d00aee290c458d906c18db2fda3dd9",
    "price": 0.089505,
    "quantity": 0.01,
    "filled_quantity": 0.01,
    "order_type": "MARKET_SELL",
    "status": "FILLED",
    "created_time": "2018-01-18T07:36:02.736890",
    "opened_time": null,
    "filled_time": "2018-01-18T07:36:02.922692",
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Downloading: ETH/BTC
Downloaded rows: 1


## Live Trade

In [ ]:
cash = .04
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
exchange = load_exchange(c.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
assets = [Asset(c.XRP, c.BTC)]
feed = ExchangeDataFeed(
    exchange=exchange,
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy()
experiment_name = 'default_live'

In [ ]:
live(experiment_name, exchange, balance, portfolio, feed, strategy)